# Calculating Pathway Score from RPPA Data 
For this script you will need two txt files: 

1) RPPA data table that has been log normalized (columns consist of samples, index consists of protein). 

2) Pathway predictor table (columns should consist of pathway, predictor, weight, and count) 

This script will calculate the z-score of each protein and then use the pathway predictor table to determine the pathway scores for each provided sample. 


In [1]:
#import libraries  

import os
import os.path
import sys
import pandas as pd
import numpy as np
import scipy.stats 


## Set Your Directory and Confirm File Naming Convention Matches the Script
Set my_directory to the path location of your files. 
Files should be in the following format RPPA_Log2_{SAMPLE NAME OF YOUR CHOICE}_.txt and Pathway_Scores_{SAMPLE NAME OF YOUR CHOICE}.txt 

It is important that each experiment has its own pathway score file, so if you change something to fit your analysis, it does not impact other experimental analysis. 

In [2]:
#set a working directory
cwd = os.getcwd()


#define working sample within directory
sample_imput= input("Please Enter the File you wish to work on that you have placed in the Imputs directory ")
ls_sample = [sample_imput]

Please Enter the File you wish to work on that you have placed in the Imputs directory SMMART_EBN_RBN_5


## Import RPPA Data Frame 
Make sure that sample names are along the column headings and that proteins are along the index. We will remove spaces, hyphens, underscores, and capitalize all protein names. The dataframe will also be transposed so that proteins are the column names. 

An excel print out will give you counts of missing data for each protein and missing protein counts for each sample. 

Whenever you load a new file be sure that the delimiter/separation match the code. EG- in this case the txt file is delimited by whitespace so we pass the argument delim_whitespace = True. If it were separated by tabs we would deleted the previous argument and replace it with : sep = '\t' .If comma delimited there is no need to specify either of these arguments. 

In [9]:
# Import Data frame with log 2 normalized sample data 
for sample in ls_sample:
    df = pd.read_csv(
        f'Imputs/{sample}.txt',
        delim_whitespace=True,
        index_col = 0
        
    )

#remove spaces hyphens and underscores 
pattern = '|'.join([' ', '-', '_'])
df.index = df.index.str.replace(pattern, '')

#Change Predictor color to uppercase
df.index = df.index.str.upper()
df=df.rename_axis('Predictor')


<ipython-input-9-12795e999a0d>:12: FutureWarning: The default value of regex will change from True to False in a future version.
  df.index = df.index.str.replace(pattern, '')


In [10]:
for sample in ls_sample:
    df2 = pd.read_csv(
        f'Imputs/Pathway_Scores.txt',
        sep = '\t' #if separated by white space use 'delim_whitespace = True' 
    
    )
#Remove inconsistent characters 
df2['Predictor'].replace(pattern,'', regex = True, inplace = True) # remove - from dataframe

#Change Predictor color to uppercase
df2['Predictor'] = df2['Predictor'].str.upper()

#change index to predictors 
df2.set_index('Predictor', inplace = True)

In [11]:
df2

,Pathway,Weight,Count
Predictor,,,
CASPASE7CLEAVEDD198,Apoptosis,1.0,1.0
CASPASE8,Apoptosis,1.0,1.0
CASPASE3CLEAVED,Apoptosis,1.0,1.0
BAK,BH3_Balance,1.0,1.0
BAX,BH3_Balance,1.0,1.0
...,...,...,...
STAT5ALPHA,JAK-STAT,1.0,1.0
JAK2,JAK-STAT,1.0,1.0
DNMT1,Epigenetic,1.0,1.0


In [12]:
df5 = pd.merge(df, df2, on='Predictor')
df5
#df5.reset_index(drop=True, inplace=True)
#df5 = df5.drop_duplicates(subset=['Protein'], keep='last')
df5 = df5.drop(['Pathway','Weight','Count'], axis=1)

In [13]:
df5

,0000254587_Gray.Lab,246633,246659,260994,261031,261071,261080,265277,265288,265297,...,TCGA-S3-AA12-01A-21-A43F-20,TCGA-S3-AA14-01A-21-A43F-20,TCGA-S3-AA15-01A-21-A43F-20,TCGA-S3-AA17-01A-21-A43F-20,TCGA-UL-AAZ6-01A-21-A43F-20,TCGA-UU-A93S-01A-11-A43F-20,TCGA-WT-AB41-01A-21-A43F-20,TCGA-XX-A899-01A-21-A43F-20,TCGA-Z7-A8R5-01A-51-A43F-20,TCGA-Z7-A8R6-01A-21-A43F-20
Predictor,,,,,,,,,,,,,,,,,,,,,
AKTPS473,-0.761440,-0.875003,0.228013,0.068184,-2.825989,-0.064495,0.362957,0.690969,1.922873,0.258439,...,-1.037200,-0.207180,-1.270800,-0.945650,-0.616830,-0.626890,-0.816150,-0.706820,-0.041003,1.112200
AKTPT308,-1.312470,-1.520390,-0.613891,-0.723969,-0.850693,-0.912850,-0.468383,-0.206265,2.130562,-0.380641,...,-0.952380,-0.332510,-0.565190,-0.879740,-0.604860,-0.638480,-1.260800,-0.701690,-0.191290,1.187000
AR,0.315681,-0.129480,3.862470,1.112446,0.189217,0.365908,1.411676,-0.424086,4.593167,2.321915,...,0.955520,2.355600,0.588770,1.380700,1.733600,0.366920,0.967810,0.382180,2.219500,1.724400
ARID1A,-0.168253,-0.324549,0.301345,0.377317,-0.268172,0.244761,0.353344,0.071541,0.188561,0.218742,...,0.359660,0.319760,-0.521370,0.255300,0.362820,0.119920,-0.081452,0.275070,0.251390,0.459090
ATMPS1981,0.558234,0.097917,-0.352625,-0.358735,0.271260,-0.248405,-0.388398,-0.252890,-0.499155,-0.286365,...,-0.075031,0.206054,0.728870,0.036875,-0.016314,0.611072,0.072553,-0.027416,0.168716,-0.400946
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WEE1PS642,0.371524,0.927216,-0.174000,-0.412384,0.870623,-0.294217,-0.467683,-0.185517,-0.386563,-0.011272,...,-0.424186,-0.047707,0.528055,-0.216135,-0.282668,-0.231589,-0.051373,0.158816,0.145272,-0.081330
YAP,-0.436937,0.137354,-0.160527,0.095887,0.052706,-0.418086,-0.058241,-0.104154,-0.281343,-0.159224,...,-0.393220,-0.275850,-0.309100,-0.610740,-0.505600,-0.338510,-0.368850,-0.207960,-0.260830,-0.480560
YAPPS127,-0.883617,0.014130,0.089790,0.169045,-0.822866,-0.668804,0.133040,0.163685,0.147866,0.053514,...,-0.767630,-0.339510,-1.470600,-1.149900,-0.768570,-0.545320,-0.564600,-0.091886,-0.369120,-0.722460


In [14]:
#transpose to get proteins in columns 
df1 = df5.T
cols = df1.columns.tolist()

#calculate missing values for proteins and samples export to view in excel 
df_samples = df1.isna().sum(axis = 1)
df_protein = df1.isna().sum(axis = 0)
df_samples.to_csv(f'Outputs/{sample}_Missing_Sample_Totals.csv', header = False)
#we write out missing proteins later but you could also write it out now
#df_protein.to_csv(f'Outputs/{sample}_Missing_Protein_Totals.csv', header = False)
df1.to_csv(f'Outputs/{sample}_score_pass.csv')

## Score pass is delt with in the R markdown document Score_Calculator this needs to be run at this point we will aslo investigate making this document irrevelent and causing it to be done in python eventually 


In [15]:
#df1=df1.transpose()

#Now we will edit the pathways 
#pathway_hold=df2.drop(['Weight','Count'], axis=1)
#df1=df1.join(pathway_hold)
#first_column = df1.pop('Pathway')
  
# insert column using insert(position,column_name,
# first_column) function
#df1.insert(0, 'Pathway', first_column)
#df1

## Deleting Rows or Columns with NaN entries CHOOSE ONLY ONE OF THE FOLLOWING per needs of experiment 
Choose option 1 if you want to delete samples that have missing protein data

Choose option 2 if you want to delete proteins that have missing values 

This step can be omitted if you do not have any NaN entries 

Add another logic gate that asks if you want to remove one or both or none do out patient dropping earlier we want to drop pathway here, if there is a protein missing in this sample we cant calculate the pathway score, if there is a protein 

WE need to drop wither the portein or the sample depeding 

In [16]:
 df1 = pd.read_csv("Outputs/Score_pass_back.csv")

In [18]:
#we will store our groups in a dictionary for looping purposes 
groups = (df1['Pathway'].append(df1['Pathway'])).unique()
print(groups)

['PI3K_Akt' 'Hormone_receptor' 'Epigenetic' 'DNA_Damage_Checkpoint'
 'Immune_Checkpoint' 'BH3_Balance' 'Hormone_signaling_Breast'
 'Tumor_Content' 'RAS_MAPK' 'G1_S' 'G0_G1' 'Apoptosis' 'Immune' 'G2_M'
 'Cell_cycle_progression' 'RTK' 'Histone_Alteration' 'DNA_Damage' 'Notch'
 'JAK-STAT' 'TSC_mTOR' 'RPA32_pS4_S8' 'RRM1' 'RRM2']


In [19]:
#We are going to use this unique column check that was submitted to stack overflow from usr yatu
def unique_cols(df):
    a = df.to_numpy() # df.values (pandas<0.24)
    return (a[0] == a).all()

In [20]:
#make a na calculator for each protein
#buoild another path for the TCGA samples
missing_proteins=df1
missing_proteins.set_index('Predictor',inplace=True)
missing_proteins=pd.DataFrame(missing_proteins.isnull().sum(axis=1))
#write this to a file 
missing_proteins.to_csv(f'Outputs/{sample}_missing_proteins.csv')

In [21]:
#Read back in because doing the missing over writes for some reaosn 
df1 = pd.read_csv("Outputs/Score_pass_back.csv")


# Depending on the option you chose in the previous set the data frame will be refered to differently

In [22]:
#This is the goal loop 
dataframe_collection = {}
while True: 
     query = input("Would you Like to Delete samples that have Missing Proteins data or would you like to delete samples that have Missing Values?")
     if query not in ['Missing Proteins',"Missing Values"]: 
        print('Please answer with Missing Proteins or Missing Values') 
    #This will remove proteins however you dont save the datasets out that needs to be fixed how do we want to do that 
     elif query== 'Missing Proteins': 
            #THis is a loop to get rid of the data in the proteins that is missing this 
            #could be modified to get rid of it in the entire dataset 
            #lets save this as a dataset of datasets
            dropped_cols=[]
            

            for i in groups:

                df_hold=df1.loc[df1.Pathway==i]
                #print(df_hold)
                #first we will check if there are any cols that are totally missing from
                #the sample set and if we find any we will get rid of them 
                ind=df_hold.columns[df_hold.isnull().values.any(axis=0)].tolist()
                #print(ind)
                #print(len(df_hold.columns))
                if len(ind)>0:
                    #print("Entered dropper")
                    df_hold=df_hold.drop(ind,axis="columns")
                #now we want to save these new datasets
                #we will do that by creating a dictionary of datasets
                dataframe_collection[i] = df_hold

                #df_droped_proteins=pd.DataFrame({"Idx":[1,i],"dfs":[df_droped_proteins,df_hold]})
                #print("count after drop")
                #print(len(df_hold.columns))
                ind.insert(0,i)
                dropped_cols.append(ind)

            droped_data=pd.DataFrame(dropped_cols)
            break
    #Now we need the one for removing samples
     elif query=="Missing Values":
       #this is approaching it from the sample droping direction
        drop_grp=[]
        for i in groups:
            df_hold=df1.loc[df1.Pathway==i]
            missing=pd.DataFrame(df_hold.isnull().sum(axis=1))
            #if the missing values are not the same across the different samples drop the sample with the most missing values.    
            #print(missing)

            while unique_cols(missing[0])==False:
                column = missing[0]
                max_value = column.idxmax()
                missing=missing.drop(labels=max_value,axis=0)
                drop_grp.append(max_value)
        print(drop_grp)
        droped_data = pd.DataFrame(df1,index = drop_grp)
        d_clean=df1.drop(drop_grp)
        for i in groups:
            df_hold=d_clean.loc[d_clean.Pathway==i]
            dataframe_collection[i] = df_hold
            #this needs to be put back through the other one to remove missing 
        break
        
print("the following samples were dropped")   
#droped_grp
print(droped_data)
droped_data.to_csv(f'Outputs/{sample}_droped_data.csv')

Would you Like to Delete samples that have Missing Proteins data or would you like to delete samples that have Missing Values?Missing Proteins
the following samples were dropped
                         0                            1    \
0                   PI3K_Akt                         None   
1           Hormone_receptor                         None   
2                 Epigenetic          0000254587_Gray.Lab   
3      DNA_Damage_Checkpoint                       265578   
4          Immune_Checkpoint          0000254587_Gray.Lab   
5                BH3_Balance                         None   
6   Hormone_signaling_Breast                         None   
7              Tumor_Content                         None   
8                   RAS_MAPK                         None   
9                       G1_S                         None   
10                     G0_G1                         None   
11                 Apoptosis  TCGA-3C-AALI-01A-21-A43F-20   
12                    Immune 

In [23]:
for key in dataframe_collection.keys():
    dataframe_collection[key].set_index(['Predictor','Pathway'],inplace=True)

In [24]:
#If you want to look at the output uncomment this 

for key in dataframe_collection.keys():
        print("\n" +"="*40)
        print(key)
        print("-"*40)
        print(dataframe_collection[key])


PI3K_Akt
----------------------------------------
                              0000254587_Gray.Lab    246633    246659  \
Predictor           Pathway                                             
AKTPS473            PI3K_Akt            -0.761440 -0.875003  0.228013   
AKTPT308            PI3K_Akt            -1.312470 -1.520390 -0.613891   
GSK3ALPHABETAPS21S9 PI3K_Akt            -0.786754 -0.997177 -0.654158   
P27PT198            PI3K_Akt            -0.039411  0.016217 -0.025065   
                    PI3K_Akt            -0.039411  0.016217 -0.025065   
                    PI3K_Akt            -0.039411  0.016217 -0.025065   
PRAS40PT246         PI3K_Akt            -0.276136 -0.485501 -0.420709   
PTEN                PI3K_Akt             0.180911 -0.327640  0.421572   

                                260994    261031    261071    261080  \
Predictor           Pathway                                            
AKTPS473            PI3K_Akt  0.068184 -2.825989 -0.064495  0.362957   
AK

## Calculate the standard deviation for each Protein (column) in the RPPA data 
Can either mean or median center. Manipulate code on line six to change method. EG: df1[col] - df1[col].mean . . . vs df1[col] - df1[col].median  

In [25]:
#Now we will do the Z Score calculation
z_Score= pd.concat(dataframe_collection.values())
z_Score  = z_Score.sub(z_Score.median(axis=0), axis=1).div(z_Score.std(ddof=0,axis=0), axis=1)  
        
  
z_Score.to_csv(f'Outputs/{sample}z_scores.csv')

## Load in the pathway score information 
take weights and calculate the sore for the individual pathways


In [26]:
for sample in ls_sample:
    print(f'Pathway_Scores.txt')
    pathways = pd.read_csv(
        f'Imputs/Pathway_Scores.txt',
        sep = '\t' #if separated by white space use 'delim_whitespace = True' 
    
    )
#Remove inconsistent characters 
pathways['Predictor'].replace(pattern,'', regex = True, inplace = True) # remove - from dataframe

#Change Predictor color to uppercase
pathways['Predictor'] = pathways['Predictor'].str.upper()


#change index to predictors 
pathways.set_index('Predictor', inplace = True)
pathways


Pathway_Scores.txt


,Pathway,Weight,Count
Predictor,,,
CASPASE7CLEAVEDD198,Apoptosis,1.0,1.0
CASPASE8,Apoptosis,1.0,1.0
CASPASE3CLEAVED,Apoptosis,1.0,1.0
BAK,BH3_Balance,1.0,1.0
BAX,BH3_Balance,1.0,1.0
...,...,...,...
STAT5ALPHA,JAK-STAT,1.0,1.0
JAK2,JAK-STAT,1.0,1.0
DNMT1,Epigenetic,1.0,1.0


we need to split out z_score again


In [27]:
#prepare to merge in score back into a format for scoring 
z_Score.reset_index(level=1, inplace=True)
z_Score
z_Score.to_csv(f'Outputs/{sample}_pathway_Zscore_data.csv')

In [28]:
#See if we are missing anything in the weights file 
rows=z_Score.index.tolist()
not_found=[]

for index, row in pathways.iterrows():
    if index in rows:
        pass
    else:
        not_found.append([index,row[0]])
            

not_found_df= pd.DataFrame(not_found)

#we will write this out for the report
not_found_df.to_csv(f'Outputs/{sample}_Scores_not_found.csv')

In [29]:
#Store the zscore in a way the loop can take it in 
for i in groups:
            df_hold=z_Score.loc[z_Score.Pathway==i]
            df_hold.dropna(axis=1, how='all')
            dataframe_collection[i] = df_hold

In [30]:
#Reset the index and check the dataformat
for key in dataframe_collection.keys():
    dataframe_collection[key].set_index('Pathway',append=True,inplace=True)
#for key in dataframe_collection.keys():
        #print("\n" +"="*40)
       # print(key)
       # print("-"*40)
       # print(dataframe_collection[key])

# Calculate the Pathway score for each of the samples 

In [31]:

#prepare the pathways for the loop 
pathway_collection={}
for i in groups:
        df_hold=pathways.loc[pathways.Pathway==i]
        pathway_collection[i] = df_hold
        
#for key in pathway_collection.keys():
        #print("\n" +"="*40)
        #print(key)
        #print("-"*40)
        #print(pathway_collection[key])

In [32]:
count=0
test={}
hold={}

for key in dataframe_collection.keys():
    cols=dataframe_collection[key].columns.tolist()

    for col in cols:
        weighted_score_path=0
       # print("entering: "+ key+" "+ col)
        #print(weighted_score_path)
        pathway_weight=0
        for i in dataframe_collection[key][col].iteritems():
            x=i[0]
            protein_score=x[0]
            
            score=i[1]
            #print(protein_score)
            #print("SCoRE")
            #print(score)
            #pathway_weight=0
            for i in pathway_collection[key]["Weight"].iteritems():
                weight=i[1]
                protein_weight=i[0]
                #print(protein_weight)
                if protein_score==protein_weight:
                    for y in pathway_collection[key]["Count"].iteritems():
                        total=y[1]
                        #print(total)
                        id_count=y[0]
                        #print(protein_weight)
                        #print(weight)
                        if protein_weight==id_count:
                            pathway_weight=pathway_weight+total
                            #print(pathway_weight)
                    weighted_score=0
                    weighted_score=(score*weight)
                    weighted_score_path+=weighted_score
        #print("pathway score for: "+ key+" "+ col)    
        #print( pathway_weight)
        #print(weighted_score_path)
        if pathway_weight==0:
            t=0
            print("ERROR IN: "+key+" pathway weight" )
        else:
            pathway_score=weighted_score_path/pathway_weight
            #print(pathway_score)
        #hold[key]={col:weighted_score_path}
            hold.setdefault(key, [])
            hold[key].append([col,pathway_score])
        

        
       # print("score for: "+ key+" "+ col)
        #print(weighted_score_path)
        #test[col]=weighted_score_path
        
    

In [33]:
#Save weighted scores to file and output 
dlist = [[key] + i for key,value in hold.items() for i in value] 
score_output = pd.DataFrame(dlist, columns=['Pathway','Sample','Pathway_score'])
score_output=score_output.pivot(index="Pathway",columns="Sample")
score_output.to_csv(f'Outputs/{sample}_pathway_score_data.csv')

In [34]:
score_output

Pathway_score                                \
Sample                   0000254587_Gray.Lab    246633    246659    260994   
Pathway                                                                      
Apoptosis                          -0.819705 -0.199588 -0.046209 -0.371845   
BH3_Balance                         0.107763 -0.081097 -0.165690 -0.058924   
Cell_cycle_progression             -0.128857 -0.404992 -0.690929  0.032787   
DNA_Damage                               NaN       NaN       NaN       NaN   
DNA_Damage_Checkpoint               0.571524  0.842819 -0.178021 -0.560250   
Epigenetic                               NaN       NaN       NaN       NaN   
G0_G1                               0.818513  0.957867  0.610706  0.051522   
G1_S                               -1.020521 -1.346071 -0.228680  0.026036   
G2_M                                0.083759 -0.379434 -0.915769  0.314079   
Histone_Alteration                 -0.484134 -0.316520  0.163623  0.236245   
Hormone_receptor                    1.735671  0.615237  3.088928  2.493613   
Hormone_signaling_Breast            0.517218 -0.037806 -0.036780  1.124872   
Immune                                   NaN -0.026430  1.005398 -0.614781   
Immune_Checkpoint                        NaN       NaN       NaN       NaN   
JAK-STAT                           -0.653303 -0.531424  0.426158  0.202250   
Notch                               0.144013  0.038288  0.135247 -0.112218   
PI3K_Akt                           -0.441937 -0.459068 -0.296189 -0.413728   
RAS_MAPK                            0.054983  0.498210  0.223225 -0.067719   
RPA32_pS4_S8                       -0.534523 -1.034405  0.132074  0.000000   
RRM1                                     NaN       NaN       NaN       NaN   
RRM2                                     NaN       NaN       NaN       NaN   
RTK                                 0.420698  0.360139  0.276506 -0.363688   
TSC_mTOR                            0.640635 -0.131664 -0.145798 -0.461620   
Tumor_Content                       0.318067 -0.790471 -0.049001  0.345285   

                                                                            \
Sample                      261031    261035    261071    261080    265277   
Pathway                                                                      
Apoptosis                -0.178160  0.178745 -0.253020 -0.307782  0.739616   
BH3_Balance               0.039147 -0.395134 -0.004602  0.021010 -0.082074   
Cell_cycle_progression   -0.687366 -0.601603  0.310879 -0.525896 -0.990466   
DNA_Damage                     NaN  0.129256       NaN       NaN       NaN   
DNA_Damage_Checkpoint     0.369786  0.188036 -0.441003 -0.358195 -0.092633   
Epigenetic                     NaN  0.732708       NaN       NaN       NaN   
G0_G1                     1.075460  0.317137  0.082373  0.193035  0.797795   
G1_S                     -1.449253  0.561255 -0.641792  0.039733 -0.428480   
G2_M                     -0.817133 -0.912393  0.687839 -0.758511 -1.268388   
Histone_Alteration        0.052622  0.465145 -0.007401  0.192666  0.323655   
Hormone_receptor          0.606779 -0.063801  3.265563  3.236556  1.626953   
Hormone_signaling_Breast  0.162787  1.394507  1.096901  1.168121  0.659303   
Immune                    0.432472  2.240432  0.649531 -0.173825  0.388719   
Immune_Checkpoint              NaN -0.564306       NaN       NaN       NaN   
JAK-STAT                 -0.475162  0.901278  0.703309  0.360326  0.391954   
Notch                     0.295950 -0.090558 -0.141612 -0.023680  0.132369   
PI3K_Akt                 -0.207400  0.340490 -0.322648 -0.150777 -0.053569   
RAS_MAPK                  0.145822  0.051121  0.308021  0.328393  0.425056   
RPA32_pS4_S8             -0.509132 -0.111895 -0.642836 -0.066279 -0.080951   
RRM1                           NaN -0.085629       NaN       NaN       NaN   
RRM2                           NaN -0.405582       NaN       NaN       NaN   
RTK                      -0.017461  0.370083 -0.2188